# MCP Calculator Client with Mistral AI (SSE Version)

This notebook demonstrates how to use MCP (Model Context Protocol) with Mistral AI's chat completion API using Server-Sent Events (SSE) connection.

## What is SSE?
Server-Sent Events (SSE) is a technology that allows a server to push data to web clients over HTTP. In the context of MCP, it provides a way to connect to MCP servers through HTTP endpoints instead of STDIO.

## Prerequisites

Before running this notebook, make sure you have:
1. The required packages installed
2. An MCP server running on `http://localhost:9321/sse`

In [1]:
# Install required packages
# !pip install mcp mistralai

## Import Required Libraries

The SSE client allows us to connect to MCP servers over HTTP, which is useful for:
- Remote server connections
- Web-based integrations
- Scenarios where STDIO is not suitable

In [6]:
import asyncio
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client
from mcp.client.stdio import stdio_client
# from mistralai import Mistral
import openai
import sys


import os
from dotenv import load_dotenv
# Load environment variables
load_dotenv()


env_vars_to_clear = ['OPENAI_API_KEY', 'OPENAI_BASE_URL', 'OPENAI_API_BASE']
for var in env_vars_to_clear:
    if os.getenv(var):
        print(f"⚠️  Removing conflicting {var}")
        del os.environ[var]


os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_ROUTER_KEY")
os.environ['OPENAI_API_BASE'] = 'https://openrouter.ai/api/v1'
os.environ['OPENAI_BASE_URL'] = 'https://openrouter.ai/api/v1'



# env_vars_to_clear = ['OPENAI_API_KEY', 'OPENAI_BASE_URL', 'OPENAI_API_BASE']
# for var in env_vars_to_clear:
#     if os.getenv(var):
#         print(f"⚠️  Removing conflicting {var}")
#         del os.environ[var]
# os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_KEY")




⚠️  Removing conflicting OPENAI_API_KEY
⚠️  Removing conflicting OPENAI_BASE_URL
⚠️  Removing conflicting OPENAI_API_BASE


## Configure Mistral AI Client

Set up your Mistral AI API key to enable LLM interactions.

## Connect to MCP Server via SSE

Unlike the STDIO version, here we connect to an HTTP endpoint. The server should be running and accessible at the specified URL.

**Note**: Make sure your MCP server is running on port 9321 before executing this cell.

In [2]:
# SSE server endpoint
server_params = "http://localhost:9321/sse"

# Connect using SSE client
async with sse_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the session
        await session.initialize()
        
        # Discover available tools
        tools_result = await session.list_tools()
        print(f"\n🧮 Calculator ready! Found {len(tools_result.tools)} tools")


🧮 Calculator ready! Found 2 tools


## Transform Tools for Mistral AI

Just like in the STDIO version, we need to convert the MCP tool format to match Mistral's expectations. The transformation process remains the same regardless of the connection type.

In [3]:
mcp_tools = []
for tool in tools_result.tools:
    tool_def = {
        "type": "function",  # Required wrapper
        "function": {        # Function definition
            "name": tool.name,
            "description": tool.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "First number"
                    },
                    "b": {
                        "type": "number",
                        "description": "Second number"
                    }
                },
                "required": ["a", "b"]
            }
        }
    }
    mcp_tools.append(tool_def)

## Display Available Tools

Let's inspect the tools that were discovered and transformed:

In [4]:
# Display the transformed tools
mcp_tools

[{'type': 'function',
  'function': {'name': 'add',
   'description': 'Add two numbers together',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number', 'description': 'First number'},
     'b': {'type': 'number', 'description': 'Second number'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'subtract',
   'description': 'Subtract b from a',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number', 'description': 'First number'},
     'b': {'type': 'number', 'description': 'Second number'}},
    'required': ['a', 'b']}}}]

## Request Calculation from Mistral

Send a calculation request to Mistral. The AI will analyze the request and select the appropriate tool.

In [9]:
user_input = "Find 5 minus 2"

# Request Mistral to perform the calculation
response = openai.chat.completions.create(
    model="openai/gpt-4o",
    messages=[{
        "role": "system",
        "content": "You are a calculator assistant. Use the add or subtract functions to help the user with calculations."
    }, {
        "role": "user",
        "content": user_input
    }],
    tools=mcp_tools,
    tool_choice="auto"
)

## Inspect MCP's Response

Let's examine what MCP returned, including the tool call details:

In [10]:
# Display the response message
response.choices[0].message

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_L9RdekeJ8apFXDarIgEoOG1u', function=Function(arguments='{"a":5,"b":2}', name='subtract'), type='function', index=0)], reasoning=None)

## Execute the Tool Call via SSE

Now we'll execute the tool that Mistral selected. The main difference from STDIO is that we're connecting via HTTP/SSE instead of process pipes.

In [11]:
message = response.choices[0].message

# Connect to SSE server to execute the tool
async with sse_client("http://localhost:9321/sse") as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize session
        await session.initialize()
        
        if message.tool_calls is not None:
            for tool_call in message.tool_calls:
                # Extract function name
                func_name = tool_call.function.name
                
                # Parse arguments (handle both string and dict formats)
                if isinstance(tool_call.function.arguments, str):
                    args = json.loads(tool_call.function.arguments)
                else:
                    args = tool_call.function.arguments
                
                # Execute the MCP tool
                result = await session.call_tool(func_name, args)
                print(f"Result: {result.content[0].text}")
        else:
            print("No tool calls found in the response.")

Result: 3.0


## Summary and Comparison

### SSE vs STDIO

**SSE (This notebook):**
- Connects via HTTP endpoint
- Good for remote servers
- Suitable for web integrations
- Requires server to be running on a specific port

**STDIO (Previous notebook):**
- Connects via process pipes
- Good for local servers
- Better process isolation
- Server runs as a subprocess

Both approaches achieve the same result but are suited for different deployment scenarios.